# We will do the prediction by loading the model which we trained earlier

In [11]:
import pandas as pd 
import nltk
import string 
import re 
from sklearn.feature_extraction.text import TfidfVectorizer  
import warnings

warnings.filterwarnings('ignore')

wnLemmatizer = nltk.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')

In [12]:
df = pd.read_csv("c:/Users/HP/Dropbox/Datasets/SMSSpamCollection.tsv" , sep = '\t')  
df.columns = ['label' , 'body_text']


def get_length_text (text ) :
    return len(text) - text.count(" ") 
def get_punc_percent (text) :
    punc_length = sum([1 for char in text if char in string.punctuation ])
    return   round( punc_length / (len(text) - text.count(' ')) , 3 ) *100 

def clean_text (text ) :
    no_punctuation = "".join ([char for char in text if char not in string.punctuation ])
    tokenized = re.split (r'[\W]+' , no_punctuation.lower() )
    lemmatized = [wnLemmatizer.lemmatize(word) for word in tokenized if word not in stopwords] 
    return lemmatized 

vectorizer = TfidfVectorizer(analyzer = clean_text)
vectorizer.fit(df["body_text"])


TfidfVectorizer(analyzer=<function clean_text at 0x00000268E2213E18>,
        binary=False, decode_error='strict', dtype=<class 'numpy.float64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), norm='l2',
        preprocessor=None, smooth_idf=True, stop_words=None,
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [13]:
from sklearn.externals import joblib 

gbmodel =  joblib.load('GradientBoosing.pkl')


In [14]:
def FilterYourMail (text) : 
    length = get_length_text(text)
    punc_percent = get_punc_percent(text)
    text_series_data = pd.Series(text)  # passing series data because df['body_text']  which we passed  
#   while fitting vectorizer is a series data , not  a dataframe .

    final_matrix = vectorizer.transform(text_series_data).toarray()
    features = pd.DataFrame({'text_length' : [length] , 'punc_percent' : [punc_percent]  })
    final_X = pd.concat([features , pd.DataFrame(final_matrix)] , axis = 1 )
    prediction =  gbmodel.predict(final_X)[0]
    if (prediction == 'ham') : return "--- It's not a spam message ---  "
    else : return "--- It's a spam message --- "


In [15]:
message = """Your free subscription is waiting to be collected. Simply text the password MIX to 85069 to verify.
  Get Usher and Britney. FML, PO Box 5249, MK17 92H. 450Ppw 16"""

print (FilterYourMail(message ) )

--- It's a spam message --- 


In [16]:
message = "Are you chilling ? "

print(FilterYourMail(message) )

--- It's not a spam message ---  


In [17]:
message = """WINNER!! As a valued network customer you have been selected to receive a $900 prize reward!
        To claim call 09061701461. Claim code KL341. Valid 12 hours only. """

print(FilterYourMail(message))

--- It's a spam message --- 


In [18]:
message = """Send me your contact number and don't forget to include what was missing in the first place.
            This behaviour of yours is not commendable . """

print(FilterYourMail(message))

--- It's not a spam message ---  
